In [29]:
#import Pkg; Pkg.add("PyFormattedStrings")

   Resolving package versions...
   Installed PyFormattedStrings ─ v0.1.10
    Updating `C:\Users\mitha\.julia\environments\v1.7\Project.toml`
  [5f89f4a4] + PyFormattedStrings v0.1.10
    Updating `C:\Users\mitha\.julia\environments\v1.7\Manifest.toml`
  [5f89f4a4] + PyFormattedStrings v0.1.10
Precompiling project...
  ✓ PyFormattedStrings
  1 dependency successfully precompiled in 33 seconds (146 already precompiled)


In [2]:
import DataStructures
using PyFormattedStrings

mutable struct Info
    iterations::Int64
    observedString::String
    possibleObservableStates
    possibleHiddenStates
    transitionDict::DataStructures.DefaultOrderedDict
    emissionDict::DataStructures.DefaultOrderedDict
end

mutable struct BaumWelch
    p::Info
    pathProb
    initialProbabilities
    outcome
    initialBackwardProbabilities
    backwardPathProbabilities
    eStepResult
end

function forward(bw::BaumWelch)
    #println(bw.pathProb)
    #initialProbabilities = DataStructures.DefaultOrderedDict(0)
    pathProb = []
    for i in 1:length(bw.p.possibleHiddenStates)
        bw.initialProbabilities[bw.p.possibleHiddenStates[i]] = 1/ length(bw.p.possibleHiddenStates) * bw.p.emissionDict[bw.p.observedString[1] * bw.p.possibleHiddenStates[i]]
    end
    push!(pathProb, bw.initialProbabilities)
    for t in 2:length(bw.p.observedString)
        newPathProbabilities = DataStructures.DefaultOrderedDict(0)
        for hiddenstate in bw.p.possibleHiddenStates
            tempProbDict = DataStructures.DefaultOrderedDict(0)
            for (k,v) in pathProb[t-1]
                tempProbDict[k * hiddenstate] = v * bw.p.transitionDict[k*hiddenstate] * bw.p.emissionDict[bw.p.observedString[t]*hiddenstate]
            end
            newPathProbabilities[hiddenstate] = sum(values(tempProbDict))
        end
        push!(pathProb, newPathProbabilities)
    end
    #outcome = 0 
    for v in values(pathProb[end])
        bw.outcome = bw.outcome + v
    end
    return pathProb
end
function backward(bw::BaumWelch)    
    bw.backwardPathProbabilities = []
    reversedObservedString = reverse(bw.p.observedString)
    #initialBackwardProbabilities = DataStructures.DefaultOrderedDict(0)
    for i in 1:length(bw.p.possibleHiddenStates)
        bw.initialBackwardProbabilities[bw.p.possibleHiddenStates[i]] = 1
    end
    push!(bw.backwardPathProbabilities, bw.initialBackwardProbabilities)
    for t in 1:length(reversedObservedString)-1
    newPathProb = DataStructures.DefaultOrderedDict(0)
        for hiddenstate in bw.p.possibleHiddenStates
            tempProbDict2 = DataStructures.DefaultOrderedDict(0)
            for (k,v) in bw.backwardPathProbabilities[t]
               tempProbDict2[k*hiddenstate] = v * bw.p.transitionDict[hiddenstate*k] * bw.p.emissionDict[reversedObservedString[t]*k]
            end
            newPathProb[hiddenstate] = sum(values(tempProbDict2))
        end
        push!(bw.backwardPathProbabilities, newPathProb)
    end
    bw.backwardPathProbabilities = reverse(bw.backwardPathProbabilities)
    return bw.backwardPathProbabilities
end

function edgeResponsibilityMatrix(bw::BaumWelch)
    #eStepResult = DataStructures.DefaultOrderedDict(0)
    for (k,v) in bw.p.transitionDict
        bw.eStepResult[k] = []
    end
    for i in 1:length(bw.pathProb)-1
        #println("i" , i)
        for k in keys(bw.eStepResult)
            j = string(k[1])
            l = string(k[end])      #68?
            value = (bw.pathProb[i][j] * bw.backwardPathProbabilities[i+1][l] * bw.p.transitionDict[j*l] * bw.p.emissionDict[bw.p.observedString[i+1]*l] / bw.outcome)
            push!(bw.eStepResult[j*l], value)
        end
    end
    newTransitionMatrix = DataStructures.DefaultOrderedDict(0)
    for (k,v) in bw.eStepResult
        newTransitionMatrix[k] = sum(v)
    end
    #print("tranistion", newTransitionMatrix)
    return newTransitionMatrix
    
end
function nodeResponsibilityMatrix(bw::BaumWelch)
    result = DataStructures.DefaultOrderedDict(0)
    for k in keys(bw.p.emissionDict)
        result[k] = []
    end
    for i in 1:length(bw.pathProb)
        for (k,v) in bw.pathProb[i]
            #println(result[bw.p.observedString[i]*k])
            push!(result[bw.p.observedString[i]*k] , v*bw.backwardPathProbabilities[i][k] / bw.outcome)
        end
    end
    newEmissionMatrix = DataStructures.DefaultOrderedDict(0)
    for (k,v) in result
        newEmissionMatrix[k] = sum(v)
    end
    #println("emission", newEmissionMatrix)
    return newEmissionMatrix
end

function parameterEstimator(bw::BaumWelch)
    newEmissionMatrix = nodeResponsibilityMatrix(bw)
    #println(newEmissionMatrix)
    newTransitionMatrix = edgeResponsibilityMatrix(bw)
    #println(newTransitionMatrix)
    transitionEstimator = DataStructures.DefaultOrderedDict(0)
    total = DataStructures.DefaultOrderedDict(0)
    for i in bw.p.possibleHiddenStates
        total[i] = 0
    end
    for (k,v) in newTransitionMatrix
        total[k[1]] += v 
    end
    for (k,v) in newTransitionMatrix
        transitionEstimator[k] = v/total[k[1]]
    end
    #println(transitionEstimator)
    
    emissionEstimator =DataStructures.DefaultOrderedDict(0)
    total1 = DataStructures.DefaultOrderedDict(0)
    for i in bw.p.possibleHiddenStates
        total1[i] = 0
    end
    for (k,v) in newEmissionMatrix
        total1[k[end]] += v
    end
    for (k,v) in newEmissionMatrix
        emissionEstimator[k] = v/ total1[k[end]]
    end
    return transitionEstimator, emissionEstimator 
end

function training(bw::BaumWelch)
   for i in 1:bw.p.iterations
        bw.pathProb = forward(bw)
        bw.backwardPathProbabilities = backward(bw)
        bw.p.transitionDict, bw.p.emissionDict = parameterEstimator(bw)
    end    
end

function print(bw::BaumWelch)
    transitionList = [bw.p.possibleHiddenStates]
    for i in range(start = 1, stop = length(bw.p.possibleHiddenStates), step = 1)
        tempList = [bw.p.possibleHiddenStates[i]]
        for j in range(start = 1, stop = length(bw.p.possibleHiddenStates), step = 1)
            push!(tempList , f"{round(bw.p.transitionDict[bw.p.possibleHiddenStates[i]*bw.p.possibleHiddenStates[j]], digits = 3):.3f}")
        end
        push!(transitionList , tempList)
    end  
    println("\t", join(transitionList[1], "\t"))
    for i in range(start = 2, stop = length(transitionList))
            println(join(transitionList[i], "\t"))
    end
    println("--------")
    emissionList = [bw.p.possibleObservableStates]
    for i in range(start = 1, stop = length(bw.p.possibleHiddenStates), step = 1)
        tempList = [bw.p.possibleHiddenStates[i]]
        for j in range(start = 1, stop = length(bw.p.possibleObservableStates), step = 1)
            push!(tempList , f"{round(bw.p.emissionDict[bw.p.possibleObservableStates[j]*bw.p.possibleHiddenStates[i]], digits = 3):.3f}")
        end
        push!(emissionList, tempList)
    end
    println("\t", join(emissionList[1], "\t"))
    for i in range(start = 2, stop = length(emissionList))
        println(join(emissionList[i], "\t"))
    end
end
    
function main()
    input = []  
    open("rosalind_ba10k_762_9_dataset.txt") do f
    # read till end of file
      while ! eof(f) 
         # read a new / next line for every iteration          
         s = readline(f) 
         push!(input, s) 
      end
    end
    iterations = parse(Int64, input[1])
    observedString = input[3]
    alphabets = input[5]
    E = [x for x in split(alphabets) if length(x) > 0]
    state = input[7]
    states = [x for x in split(state) if length(x) > 0]
    tempSpace = []
    start = 0
    for ln in input[8:end]
        if start == 1
            if startswith(ln, "---")
                start = 0 
                continue
            end
            push!(tempSpace, ln)
        end
        if startswith(ln, "---")
            start = 1
        end
    end
    transition = []
    for i in tempSpace
        r = i
        s = [x for x in split(r) if length(x) > 0]
        push!(transition, s)
    end
    transitionDict = DataStructures.DefaultOrderedDict(0)
    for i in 1:length(states)
        for j in 2:length(states)+1
            transitionDict[transition[1][i] * transition[j][1]] = parse(Float64, transition[i + 1][j])
        end
    end
    
    tempSpace2 = []
    start = 0
    for ln in input[10:end]
        if start == 1
            if startswith(ln, "---")
                start = 0
                continue
            end
            push!(tempSpace2, ln)
        end
        if startswith(ln, "---")
            start = 1
        end
    end
    emission = []
    for i in tempSpace2
        r = i
        s = [x for x in split(r) if length(x) >0]
        push!(emission, s)
    end
    emissionDict = DataStructures.DefaultOrderedDict(0)
    for i in 1:length(E)
        for j in 2:length(states)+1
            emissionDict[emission[1][i] * emission[j][1]] = parse(Float64, emission[j][i + 1])
        end
    end
    #println(observedString)
    @time begin
    p = Info(iterations, observedString, E, states, transitionDict, emissionDict)
    #BaumWelch(p)
    pathProb = []
    initialProbabilities = DataStructures.DefaultOrderedDict(0)
    outcome = 0
    backwardPathProbabilities = []
    initialBackwardProbabilities = DataStructures.DefaultOrderedDict(0)
    eStepResult = DataStructures.DefaultOrderedDict(0)
    bw = BaumWelch(p, pathProb, initialProbabilities, outcome, initialBackwardProbabilities, backwardPathProbabilities, eStepResult)
    
    training(bw)
    end
    print(bw)
end   
main()


  1.494969 seconds (12.45 M allocations: 423.407 MiB, 5.51% gc time, 0.81% compilation time)
	A	B	C	D
A	0.000	0.087	0.232	0.680
B	0.566	0.434	0.000	0.000
C	0.442	0.124	0.140	0.294
D	0.001	0.401	0.548	0.050
--------
	x	y	z
A	0.000	0.067	0.933
B	0.663	0.000	0.337
C	0.000	0.975	0.025
D	0.989	0.000	0.011
